# Query Rewrite
- 질문 복잡성이 높을 때
- 앞단에 라우터 걸어서 복잡성 높은 쿼리에 대한 셀렉터 모듈로 활용될 수 있는 구조

In [ ]:
!pip install llama_index openai datasets llama-index-retrievers-bm25

In [ ]:
# set up OpenAI
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:


import logging
import sys


from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine
from IPython.display import Markdown, display
import pprint
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

#글로벌 세팅
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small"
)
Settings.llm= OpenAI(model='gpt-4o-mini')

In [ ]:
# Dataset 로드
from datasets import load_dataset

ds = load_dataset("HAERAE-HUB/KOREAN-WEBTEXT", split='train[:20]')
data = ds.to_pandas()

In [ ]:
data

In [ ]:
# Document 오브젝트로 변환
from llama_index.core import Document, VectorStoreIndex
docs = []

#Iterative하게 Document 만들기
for i, row in data.iterrows():
    docs.append(Document(
        text=row['text'],
        # extra_info={'title': row['title']}
    ))

In [ ]:
index = VectorStoreIndex.from_documents(
    docs
)

## 1. 커스텀 프롬프트 활용

In [ ]:
from llama_index.core import PromptTemplate
from llama_index.llms.openai import OpenAI


# 쿼리 분해 프롬프트 작성
query_gen_str = """
너는 사용자가 대충 쓴 질문에 대해서, 최대한 답변하기 위한 근거를 찾기 위한 다수의 서치 쿼리를 생성해 내야해.
생성하는 쿼리 중에 딱 하나는 역으로 된 질문으로 추가해서, 상대적 근거로 활용될 수 있도록 해야해.
{num_queries}개의 서치 쿼리를 만들어 내고, 하나당 한줄씩 사용해.
Query: {query}
Queries:
"""

# 프롬프트템플릿화
query_gen_prompt = PromptTemplate(query_gen_str)

#쿼리 브레이커에 사용할 llm 정의 - 최고의 성능모델이 아니어도 됨
llm =

# 커스텀 제너레이터
def generate_queries(llm, query: str, num_queries: int = 4):
    response = llm.predict(
        query_gen_prompt, num_queries=num_queries, query=query
    )
    queries = response.split("\n")
    queries_str = "\n".join(queries)
    print(f"Generated queries:\n{queries_str}")
    return queries

In [ ]:
# 잘 답변못했던 질문 확인
queries =

In [ ]:
queries

In [ ]:
from tqdm.asyncio import tqdm
import nest_asyncio
nest_asyncio.apply()

# 다수의 쿼리를 동시에 날려야 하기 떄문에 async 사용
# 사용할 다수 Retriever들의 결과를 합치는 용도
async def run_queries(queries, retrievers):
    tasks = []
    for query in queries:
        for i, retriever in enumerate(retrievers):
            tasks.append()

    task_results = await tqdm.gather(*tasks)

    results_dict = {}
    for i, (query, query_result) in enumerate(zip(queries, task_results)):
        results_dict[(query, i)] = query_result

    return results_dict

In [ ]:

from llama_index.retrievers.bm25 import BM25Retriever


## vector retriever - top 2
vector_retriever =

## bm25 retriever - top 2
bm25_retriever = BM25Retriever.from_defaults(
    docstore=index.docstore, similarity_top_k=2
)

In [ ]:
# 쿼리 실행
results_dict = await run_queries(queries, [vector_retriever, bm25_retriever])

In [ ]:
results_dict

In [ ]:
from typing import List
from llama_index.core.schema import NodeWithScore

# 여러 Retriever 메서드 사용했기 때문에 결과 퓨전 필요
# 기본적인 rrf 사용
def fuse_results(results_dict, similarity_top_k: int = 2):
    """Fuse results."""
    k = 60.0
    fused_scores = {}
    text_to_node = {}

    for nodes_with_scores in results_dict.values():
        for rank, node_with_score in enumerate(
            sorted(
                nodes_with_scores, key=lambda x: x.score or 0.0, reverse=True
            )
        ):
            text = node_with_score.node.get_content()
            text_to_node[text] = node_with_score
            if text not in fused_scores:
                fused_scores[text] = 0.0
            fused_scores[text] += 1.0 / (rank + k)

    # fusion 스코어 기반 소팅
    reranked_results = dict(
        sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    )
    reranked_nodes: List[NodeWithScore] = []
    for text, score in reranked_results.items():
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score

    return reranked_nodes[:similarity_top_k]

In [ ]:
# 결과 퓨징
final_results =

In [ ]:
for n in final_results:
    print(n.score, "\n", n.text, "\n********\n")

In [ ]:
from typing import List

from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore
import asyncio

# 퓨전리트리버 클래스 정의
class FusionRetriever(BaseRetriever):

    def __init__(
        self,
        llm,
        retrievers: List[BaseRetriever],
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._retrievers = retrievers
        self._similarity_top_k = similarity_top_k
        self._llm = llm
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        queries = generate_queries(
            self._llm, query_bundle.query_str, num_queries=4
        )
        results = asyncio.run(run_queries(queries, self._retrievers))
        final_results = fuse_results(
            results, similarity_top_k=self._similarity_top_k
        )

        return final_results

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

#퓨전리트리버 생성
fusion_retriever = FusionRetriever(

)
# 퓨전리트리버를 쿼리엔진으로 사용
query_engine =

In [ ]:
query_str=

In [ ]:
#naive
naive_query_engine =
response =


In [ ]:
print(str(response))

In [ ]:
# 퓨전리트리버 답안
response2 = query_engine.query(query_str)

In [ ]:
print(str(response2))